In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
example = "My name is Sylvian and I work at Hugging Face in Brooklyn."
encoding = tokenizer(example)
type(encoding)

c:\Users\TYC\Desktop\python code\hugging face\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


transformers.tokenization_utils_base.BatchEncoding

In [2]:
tokenizer.is_fast

True

In [3]:
print(encoding.tokens())
encoding.word_ids()

['[CLS]', 'My', 'name', 'is', 'Sylvia', '##n', 'and', 'I', 'work', 'at', 'Hu', '##gging', 'Face', 'in', 'Brooklyn', '.', '[SEP]']


[None, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, None]

In [4]:
start, end = encoding.word_to_chars(3)
example[start:end]

'Sylvian'

In [5]:
from transformers import pipeline

token_classifier = pipeline("token-classification")
token_classifier("My Name is Sylvian and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


[{'entity': 'I-PER',
  'score': np.float32(0.9984806),
  'index': 4,
  'word': 'Sylvia',
  'start': 11,
  'end': 17},
 {'entity': 'I-PER',
  'score': np.float32(0.99875104),
  'index': 5,
  'word': '##n',
  'start': 17,
  'end': 18},
 {'entity': 'I-ORG',
  'score': np.float32(0.9225493),
  'index': 10,
  'word': 'Hu',
  'start': 33,
  'end': 35},
 {'entity': 'I-ORG',
  'score': np.float32(0.9668133),
  'index': 11,
  'word': '##gging',
  'start': 35,
  'end': 40},
 {'entity': 'I-ORG',
  'score': np.float32(0.9762864),
  'index': 12,
  'word': 'Face',
  'start': 41,
  'end': 45},
 {'entity': 'I-LOC',
  'score': np.float32(0.9939575),
  'index': 14,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [6]:
token_classifier = pipeline("token-classification", aggregation_strategy = "simple")
token_classifier("My Name is Sylvian and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


[{'entity_group': 'PER',
  'score': np.float32(0.99861586),
  'word': 'Sylvian',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': np.float32(0.95521635),
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': np.float32(0.9939575),
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

example = "My name is Sylvian and I work at Hugging Face in Brooklyn."
inputs = tokenizer(example, return_tensors="pt")
outputs = model(**inputs)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
print(inputs["input_ids"].shape)
print(outputs.logits.shape)

torch.Size([1, 17])
torch.Size([1, 17, 9])


In [9]:
#apply softmax
import torch

probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0].tolist()
predictions = outputs.logits.argmax(dim=-1)[0].tolist()
predictions

[0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 6, 6, 6, 0, 8, 0, 0]

In [10]:
model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}

In [11]:
results = []
tokens = inputs.tokens()

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        results.append({"entity":label, "score": probabilities[idx][pred], "word": tokens[idx]})

results

[{'entity': 'I-PER', 'score': 0.9984416365623474, 'word': 'Sylvia'},
 {'entity': 'I-PER', 'score': 0.99903404712677, 'word': '##n'},
 {'entity': 'I-ORG', 'score': 0.946740984916687, 'word': 'Hu'},
 {'entity': 'I-ORG', 'score': 0.976804792881012, 'word': '##gging'},
 {'entity': 'I-ORG', 'score': 0.9843197464942932, 'word': 'Face'},
 {'entity': 'I-LOC', 'score': 0.9937911629676819, 'word': 'Brooklyn'}]

In [12]:
inputs_with_offset = tokenizer(example, return_offsets_mapping=True)
inputs_with_offset["offset_mapping"]

[(0, 0),
 (0, 2),
 (3, 7),
 (8, 10),
 (11, 17),
 (17, 18),
 (19, 22),
 (23, 24),
 (25, 29),
 (30, 32),
 (33, 35),
 (35, 40),
 (41, 45),
 (46, 48),
 (49, 57),
 (57, 58),
 (0, 0)]

In [13]:
example[49:57]

'Brooklyn'

In [14]:
tokens = inputs_with_offset.tokens()
offsets = inputs_with_offset["offset_mapping"]

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        start, end = offsets[idx]
        results.append({
            "entity": label,
            "score": probabilities[idx][pred],
            "word": tokens[idx],
            "start": start,
            "end": end, 
        })

results

[{'entity': 'I-PER', 'score': 0.9984416365623474, 'word': 'Sylvia'},
 {'entity': 'I-PER', 'score': 0.99903404712677, 'word': '##n'},
 {'entity': 'I-ORG', 'score': 0.946740984916687, 'word': 'Hu'},
 {'entity': 'I-ORG', 'score': 0.976804792881012, 'word': '##gging'},
 {'entity': 'I-ORG', 'score': 0.9843197464942932, 'word': 'Face'},
 {'entity': 'I-LOC', 'score': 0.9937911629676819, 'word': 'Brooklyn'},
 {'entity': 'I-PER',
  'score': 0.9984416365623474,
  'word': 'Sylvia',
  'start': 11,
  'end': 17},
 {'entity': 'I-PER',
  'score': 0.99903404712677,
  'word': '##n',
  'start': 17,
  'end': 18},
 {'entity': 'I-ORG',
  'score': 0.946740984916687,
  'word': 'Hu',
  'start': 33,
  'end': 35},
 {'entity': 'I-ORG',
  'score': 0.976804792881012,
  'word': '##gging',
  'start': 35,
  'end': 40},
 {'entity': 'I-ORG',
  'score': 0.9843197464942932,
  'word': 'Face',
  'start': 41,
  'end': 45},
 {'entity': 'I-LOC',
  'score': 0.9937911629676819,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

In [15]:
example[33:45]

'Hugging Face'

In [16]:
import numpy as np

results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

idx = 0

while idx < len(predictions):
    pred = predictions[idx]
    label = model.config.id2label[pred]

    if label != "O":
        #remove B- or I-
        label = label[2:]
        start, _ = offsets[idx]
        #take all I-
        all_scores = []

        while(idx < len(predictions) and model.config.id2label[predictions[idx]] == f"I-{label}"):

            all_scores.append(probabilities[idx][pred])
            _, end = offsets[idx]
            idx += 1
        
        score = np.mean(all_scores).item()
        word = example[start:end]
        results.append(
            {
                "entity_group": label,
                "score": score,
                "word": word,
                "start": start,
                "end": end,
            }
        )

    idx += 1

results

[{'entity_group': 'PER',
  'score': 0.9987378418445587,
  'word': 'Sylvian',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9692885080973307,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9937911629676819,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]